Import and Data Transforming

In [2]:
import pandas as pd
import numpy as np


In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(r"filelocation.csv")
df.head()

In [ ]:
df = df.drop(df.columns[[0, 4, 6, 11]], axis=1)
df.head()

In [ ]:
df['Quote'] = pd.to_numeric(df[df.columns[0]])
df['Sold-to'] = pd.to_numeric(df[df.columns[1]])
df['Customer'] = df[df.columns[2]]
df['Material Description'] = df[df.columns[3]]
df['Item'] = pd.to_numeric(df[df.columns[4]])
df['Part Number'] = df[df.columns[5]]
df['Qty'] = pd.to_numeric(df[df.columns[6]])
df['Price'] = pd.to_numeric(df[df.columns[7]])
df['Estimated'] = pd.to_numeric(df[df.columns[8]])
df['Cost'] = pd.to_numeric(df['Estimated']/df['Qty'])
df['Margin %'] = pd.to_numeric(df[df.columns[9]])
df['Margin'] = pd.to_numeric(df['Margin %']/100)
df = df[['Quote','Sold-to','Customer','Material Description','Item','Part Number','Qty','Price','Cost','Margin']]
df.head()

#No Inflation Output

In [ ]:
# Create Excel writer
writer = pd.ExcelWriter(r'filelocation.xlsx', engine='xlsxwriter') 
workbook = writer.book
worksheet = workbook.add_worksheet() 

worksheet.hide_gridlines(2)

# Formats
boldfill = workbook.add_format({'bold': True, 'bg_color': '#CCECFF', 'border': 1})
fill = workbook.add_format({'bg_color': '#CCECFF', 'border': 1})
header = workbook.add_format({'bold': True, 'border': 1})
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
percent_format = workbook.add_format({'num_format': '0%'})
fill_currency = workbook.add_format({'bg_color': '#CCECFF', 'border': 1, 'num_format': '$#,##0.00'})
fill_percent = workbook.add_format({'bg_color': '#CCECFF', 'border': 1, 'num_format': '0%'})

# Title
title = 'title'

worksheet.write('A1', title, boldfill)

# Write DataFrame to Excel starting at row 4 
start_row = 3
start_col = 0
df.to_excel(writer, sheet_name='Sheet1', startrow=start_row, startcol=start_col, index=False)

# prep for new columns
cost_col = df.columns.get_loc('Cost') + start_col
price_col = df.columns.get_loc('Price') + start_col
qty_col = df.columns.get_loc('Qty') + start_col
total_cost = len(df.columns) + start_col
total_price = total_cost + 1
margin_col = df.columns.get_loc('Margin') + start_col

#currency formats
worksheet.set_column(cost_col, cost_col, None, currency_format)
worksheet.set_column(price_col, price_col, None, currency_format)
worksheet.set_column(total_cost, total_cost, None, currency_format)
worksheet.set_column(total_price, total_price, None, currency_format)

#percentage formats
worksheet.set_column(margin_col, margin_col, None, percent_format)


#new columns
worksheet.write(start_row, total_cost, 'Total Cost', header)
worksheet.write(start_row, total_price, 'Total Price', header)


for i, row in enumerate(range(start_row + 1, start_row + 1 + len(df))):  # Start writing formulas after headers
    worksheet.write_formula(
        row, price_col,
        f'=${chr(64+cost_col + 1)}{row + 1}/(1 - ${chr(64+margin_col + 1)}{row + 1})'  
    )
    worksheet.write_formula(
        row, total_cost,
        f'=${chr(64+cost_col + 1)}{row + 1}*${chr(64+qty_col + 1)}{row + 1}'  
    )
    worksheet.write_formula(
        row, total_price,
        f'=${chr(64+price_col + 1)}{row + 1}*${chr(64+qty_col + 1)}{row + 1}'  
    )

# Total and GM
ext_cost_formula = f'=SUBTOTAL(9, ${chr(64+total_cost + 1)}{start_row + 2}:${chr(64+total_cost + 1)}{start_row + 1 + len(df)})'
ext_price_formula = f'=SUBTOTAL(9, ${chr(64+total_price + 1)}{start_row + 2}:${chr(64+total_price + 1)}{start_row + 1 + len(df)})'
gm_formula = f'=(O2-N2)/O2'

worksheet.write('N1', 'Ext Cost', boldfill)
worksheet.write_formula('N2', ext_cost_formula, fill_currency)

worksheet.write('O1', 'Ext Price', boldfill)
worksheet.write_formula('O2', ext_price_formula, fill_currency)

worksheet.write('P1', 'GM', boldfill)
worksheet.write_formula('P2', gm_formula, fill_percent)


#Summary Sheet
worksheet2 = workbook.add_worksheet('summary')

border = workbook.add_format({'border': 2})
boldborderfill = workbook.add_format({'bold': True, 'bg_color': '#DDD9C4', 'border': 2})

worksheet2.write('A1', 'Customer Name', border)
worksheet2.write('A2', 'Item Types', border)
worksheet2.write('A3', 'Bid Type: (AdHoc / LTA/ Renewal)', border)
worksheet2.write('A4', 'Total Lines (Individual P/Ns)', border)
worksheet2.write('A5', 'Value', border)
worksheet2.write('A6', 'Gross Margin', border)
worksheet2.write('A7', 'Price Protection', border)
worksheet2.write('A8', 'Quote IDs', border)

worksheet2.write('B1', '', boldborderfill)
worksheet2.write('B2', '', border)
worksheet2.write('B3', '', border)
worksheet2.write('B4', '', border)
worksheet2.write('B5', '', border)
worksheet2.write('B6', '', border)
worksheet2.write('B7', '', border)
worksheet2.write('B8', '', border)


writer._save()

#Inflation Output

In [ ]:
compound_periods = 5
rate = 0.05

# Create Excel writer
writer = pd.ExcelWriter(r'filelocation.xlsx', engine='xlsxwriter') 
workbook = writer.book
worksheet = workbook.add_worksheet('pricing') 

worksheet.hide_gridlines(2)

# Formats
boldfill = workbook.add_format({'bold': True, 'bg_color': '#CCECFF', 'border': 1})
fill = workbook.add_format({'bg_color': '#CCECFF', 'border': 1})
header = workbook.add_format({'bold': True, 'border': 1})
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
percent_format = workbook.add_format({'num_format': '0%'})
fill_currency = workbook.add_format({'bg_color': '#CCECFF', 'border': 1, 'num_format': '$#,##0.00'})
fill_percent = workbook.add_format({'bg_color': '#CCECFF', 'border': 1, 'num_format': '0%'})

# Title 
title = 'title'

worksheet.write('A1', title, boldfill)

# Write DataFrame to Excel starting at row 4 (row index 3)
start_row = 3
start_col = 0
df.to_excel(writer, sheet_name='pricing', startrow=start_row, startcol=start_col, index=False)

# prep for new columns
cost_col = df.columns.get_loc('Cost') + start_col
price_col = df.columns.get_loc('Price') + start_col
qty_col = df.columns.get_loc('Qty') + start_col
total_cost = len(df.columns) + start_col
total_price = total_cost + 1
future_cost_formula_col = total_price + 1
future_price_formula_col = future_cost_formula_col + 1
cost_ext_formula_col = future_price_formula_col + 1
price_ext_formula_col = cost_ext_formula_col + 1
today_margin_formula_col = price_ext_formula_col + 1
margin_col = df.columns.get_loc('Margin') + start_col

#currency formats
worksheet.set_column(cost_col, cost_col, None, currency_format)
worksheet.set_column(price_col, price_col, None, currency_format)
worksheet.set_column(total_cost, total_cost, None, currency_format)
worksheet.set_column(total_price, total_price, None, currency_format)
worksheet.set_column(future_cost_formula_col, future_cost_formula_col, None, currency_format)
worksheet.set_column(future_price_formula_col, future_price_formula_col, None, currency_format)
worksheet.set_column(cost_ext_formula_col, cost_ext_formula_col, None, currency_format)
worksheet.set_column(price_ext_formula_col, price_ext_formula_col, None, currency_format)


#percentage formats
worksheet.set_column(margin_col, margin_col, None, percent_format)
worksheet.set_column(today_margin_formula_col, today_margin_formula_col, None, percent_format)


#inflation box
worksheet.write('M1', 'Inflation Periods', boldfill)
worksheet.write('N1', compound_periods, fill)
worksheet.write('M2', 'Rate', boldfill)
worksheet.write('N2', rate, fill_percent)

#new columns
worksheet.write(start_row, total_cost, 'Total Cost', header)
worksheet.write(start_row, total_price, 'Total Price', header)
worksheet.write(start_row, future_cost_formula_col, 'Future Cost', header)
worksheet.write(start_row, future_price_formula_col, 'Future Price', header)
worksheet.write(start_row, cost_ext_formula_col, 'Future Cost Ext', header)
worksheet.write(start_row, price_ext_formula_col, 'Future Price Ext', header)
worksheet.write(start_row, today_margin_formula_col, "Today's Margin", header)

for i, row in enumerate(range(start_row + 1, start_row + 1 + len(df))):  # Start writing formulas after headers
    worksheet.write_formula(
        row, price_col,
        f'=${chr(65+cost_col)}{row + 1}/(1 - ${chr(65+margin_col)}{row + 1})'  
    )
    worksheet.write_formula(
        row, total_cost,
        f'=${chr(65+cost_col)}{row + 1}*${chr(65+qty_col)}{row + 1}'  
    )
    worksheet.write_formula(
        row, total_price,
        f'=${chr(65+price_col)}{row + 1}*${chr(65+qty_col)}{row + 1}'  
    )

    worksheet.write_formula(
        row, future_cost_formula_col,
        f'=${chr(65+cost_col)}{row + 1}*(1+$N$2)^$N$1'  
    )
    worksheet.write_formula(
        row, future_price_formula_col,
        f'=${chr(65+price_col)}{row + 1}*(1+$N$2)^$N$1' 
    )
    worksheet.write_formula(
        row, cost_ext_formula_col,
        f'=${chr(65+future_cost_formula_col)}{row + 1}*${chr(65+qty_col)}{row + 1}'  
    )
    worksheet.write_formula(
        row, price_ext_formula_col,
        f'=${chr(65+future_price_formula_col)}{row + 1}*${chr(65+qty_col)}{row + 1}'  
    )
    worksheet.write_formula(
        row, today_margin_formula_col,
        f'=(${chr(65+future_price_formula_col)}{row + 1}-${chr(65+cost_col)}{row + 1})/${chr(65+future_price_formula_col)}{row + 1}'  
    )

# Total and GM
todays_cost_ext = f'=SUBTOTAL(9, ${chr(64+total_cost + 1)}{start_row + 2}:${chr(64+total_cost + 1)}{start_row + 1 + len(df)})'
ext_cost_formula = f'=SUBTOTAL(9, ${chr(64+cost_ext_formula_col + 1)}{start_row + 2}:${chr(64+cost_ext_formula_col + 1)}{start_row + 1 + len(df)})'
ext_price_formula = f'=SUBTOTAL(9, ${chr(64+price_ext_formula_col + 1)}{start_row + 2}:${chr(64+price_ext_formula_col + 1)}{start_row + 1 + len(df)})'
gm_formula = f'=(R2-Q2)/R2'
todays_gm_formula = f'=(R2-P2)/R2'

worksheet.write('P1', "Today's Cost Ext", boldfill)
worksheet.write_formula('P2', todays_cost_ext, fill_currency)

worksheet.write('Q1', 'Future Cost Ext', boldfill)
worksheet.write_formula('Q2', ext_cost_formula, fill_currency)

worksheet.write('R1', 'Future Price Ext', boldfill)
worksheet.write_formula('R2', ext_price_formula, fill_currency)

worksheet.write('S1', 'GM', boldfill)
worksheet.write_formula('S2', gm_formula, fill_percent)

worksheet.write('T1', "Today's GM", boldfill)
worksheet.write_formula('T2', todays_gm_formula, fill_percent)



#Summary Sheet
worksheet2 = workbook.add_worksheet('summary')

border = workbook.add_format({'border': 2})
boldborderfill = workbook.add_format({'bold': True, 'bg_color': '#DDD9C4', 'border': 2})

worksheet2.write('A1', 'Customer Name', border)
worksheet2.write('A2', 'Item Types', border)
worksheet2.write('A3', 'Bid Type: (AdHoc / LTA/ Renewal)', border)
worksheet2.write('A4', 'Total Lines (Individual P/Ns)', border)
worksheet2.write('A5', 'Value', border)
worksheet2.write('A6', 'Gross Margin', border)
worksheet2.write('A7', 'Price Protection', border)
worksheet2.write('A8', 'Quote IDs', border)

worksheet2.write('B1', '', boldborderfill)
worksheet2.write('B2', '', border)
worksheet2.write('B3', '', border)
worksheet2.write('B4', '', border)
worksheet2.write('B5', '', border)
worksheet2.write('B6', '', border)
worksheet2.write('B7', '', border)
worksheet2.write('B8', '', border)



writer._save()